<a href="https://colab.research.google.com/github/jadenfang/MNIST-NN/blob/main/MNIST_NN___Skeletal_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------
# MNIST DIGIT CLASSIFIER (PyTorch)
# -----------------------------

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import gradio as gr
from PIL import ImageOps

In [ ]:
# -----------------------------
# 1. LOAD DATA
# Transforms are preprocessing steps that get applied automatically to every image
# you load from a dataset.
# Think of transforms as a recipe that says:

# “Every time you give me an image, do X, then Y, then Z to it.”
# “For every MNIST image: convert it to a PyTorch tensor.
# MNIST images come in as PIL images (Python Imaging Library).

# But your neural network expects tensors.
# -----------------------------
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
# Load training dataset (MNIST)
train_dataset = datasets.MNIST(
    root="./data",
    train=True,
    transform=transform,
    download=True
)


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 497kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.61MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.03MB/s]


In [ ]:
# Load test dataset
test_dataset = datasets.MNIST(
    root="./data",
    train=False,
    transform=transform,
    download=True
)


In [ ]:
# Make DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# TODO: Access and print the unique labels in the training data set using the train_loader object
unique_labels = set()

for images, labels in train_loader:
    for label in labels:
        unique_labels.add(label.item())

print("Unique labels in the training set:", unique_labels)


Unique labels in the training set: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [ ]:
# -----------------------------
# 2. DEFINE NEURAL NETWORK
# TODO: Design a Neural Network with 1 hidden layer of 128 neurons
# -----------------------------
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()

        # TODO: Define layers
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Flatten image: (batch, 1, 28, 28) → (batch, 784)
        x = x.view(-1, 28*28)

        # TODO: Add activation between layers
        x = torch.relu(self.fc1(x))

        # TODO: Output layer
        x = self.fc2(x)

        return x


In [ ]:
# TODO: Create the model
model = SimpleNN()


In [ ]:
# -----------------------------
# 3. LOSS FUNCTION + OPTIMIZER
# -----------------------------
# TODO: Define your loss function
criterion = nn.CrossEntropyLoss()

# TODO: Setup your gradient descent . Try different values for the learning rate
optimizer = optim.SGD(model.parameters(), lr=0.01)



In [ ]:
# -----------------------------
# 4. TRAINING LOOP
# -----------------------------

# TODO: Define the number of epochs
epochs = 100

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        # TODO: Reset the gradients
        optimizer.zero_grad()

        # TODO: Forward pass
        outputs = model(images)

        # TODO: Compute loss
        loss = criterion(outputs, labels)

        # TODO: Backpropagate
        loss.backward()

        # TODO: Update gradients
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 1164.0586
Epoch 2, Loss: 464.9220
Epoch 3, Loss: 368.1990
Epoch 4, Loss: 330.4008
Epoch 5, Loss: 307.1428
Epoch 6, Loss: 290.5471
Epoch 7, Loss: 276.7455
Epoch 8, Loss: 265.0236
Epoch 9, Loss: 254.3673
Epoch 10, Loss: 244.7503
Epoch 11, Loss: 236.0134
Epoch 12, Loss: 227.5987
Epoch 13, Loss: 220.0238
Epoch 14, Loss: 212.8399
Epoch 15, Loss: 206.0435
Epoch 16, Loss: 200.0184
Epoch 17, Loss: 194.0432
Epoch 18, Loss: 188.4332
Epoch 19, Loss: 183.2000
Epoch 20, Loss: 178.2944
Epoch 21, Loss: 173.6438
Epoch 22, Loss: 169.0456
Epoch 23, Loss: 164.8056
Epoch 24, Loss: 160.7608
Epoch 25, Loss: 156.8984
Epoch 26, Loss: 153.2747
Epoch 27, Loss: 149.7147
Epoch 28, Loss: 146.3392
Epoch 29, Loss: 142.9605
Epoch 30, Loss: 139.8703
Epoch 31, Loss: 136.9220
Epoch 32, Loss: 133.9785
Epoch 33, Loss: 131.1656
Epoch 34, Loss: 128.3928
Epoch 35, Loss: 125.8784
Epoch 36, Loss: 123.2891
Epoch 37, Loss: 120.9619
Epoch 38, Loss: 118.5405
Epoch 39, Loss: 116.4329
Epoch 40, Loss: 114.1399
Epoch 41

In [ ]:
# -----------------------------
# 5. EVALUATION
# -----------------------------
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        # TODO: Forward pass
        outputs = model(images)

        # Predicted class = index of max logit
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 97.59%


In [ ]:
# -----------------------------
# 6. TEST SINGLE PREDICTION
# -----------------------------
# -----------------------------
# 6. TEST SINGLE PREDICTION
# -----------------------------
# ------------------------------
# Gradio Sketchpad gives you:

# * a full-color NumPy array

# * black digit on white background

# * large resolution

# * no consistent scale
#
# Hence the preprocessing
# ------------------------------

def preprocess_image(image):
    sketch_transform = transforms.Compose([
    transforms.ToPILImage(),                      # NumPy → PIL
    transforms.Grayscale(),                       # ensure 1 channel
    transforms.Resize((28, 28)),                  # 28x28 like MNIST
    transforms.Lambda(lambda img: ImageOps.invert(img)),  # invert colors
    transforms.ToTensor(),                        # → tensor, shape (1,28,28), values in [0,1]
    ])
    # Gradio Sketchpad sometimes passes a dict with 'composite'
    if isinstance(image, dict):
        image = image['composite']   # this is a NumPy array

    # Apply the preprocessing transform
    img_tensor = sketch_transform(image)  # (1, 28, 28)

    # Add batch dimension → (1, 1, 28, 28)
    img_tensor = img_tensor.unsqueeze(0)

    return img_tensor

def predict_digit(image):
    # --- STEP 1: CHECK IF SOMETHING HAS BEEN DRAWN ---
    if image is None: return "Draw something!"

    # --- STEP 2: PREPROCESS THE IMAGE ---
    img_tensor = preprocess_image(image)

    # --- STEP 3: RUN THE MODEL ---
    with torch.no_grad():
        prediction = model(img_tensor)

        # Get the index of the highest score (the predicted digit)
        predicted_digit = torch.argmax(prediction).item()

    return str(predicted_digit)

# UI Setup
interface = gr.Interface(fn=predict_digit, inputs=gr.Sketchpad(label="Draw Here"), outputs="label")
interface.queue().launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1af21a0ea733d4c428.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
